In [1]:
!pip install -q langchain-core langchain-community langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
# OpenAI의 gpt-5-mini 모델을 사용하는 모델 객체 생성
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-5-mini",
    temperature=0,
)

# 표준화된 .invoke() 메서드로 모델 호출
response = llm.invoke("대한민국의 수도는 어디인가요?")
print(response.content)

대한민국의 수도는 서울특별시(서울)입니다. 다만 행정 기능 일부는 세종특별자치시로 이전되어 정부 부처 일부가 세종에 있습니다.


In [5]:
# Prompts 구성 요소 사용 예시
from langchain_core.prompts import ChatPromptTemplate

# {topic}과 {language}라는 두 개의 입력 변수를 가진 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_template(
    "{topic}에 대해 {language}로 간략히 설명해주세요."
)

# 템플릿에 변수 값을 채워 실제 프롬프트를 생성
prompt_filled = prompt_template.format_prompt(topic="인공지능", language="한국어")
print(prompt_filled.to_messages())

[HumanMessage(content='인공지능에 대해 한국어로 간략히 설명해주세요.', additional_kwargs={}, response_metadata={})]


In [6]:
# Output Parsers 구성 요소 사용 예시
from langchain_core.output_parsers import JsonOutputParser

# JSON 형식으로 답변을 생성하도록 유도하는 프롬프트
json_prompt = "다음 인물에 대한 정보를 JSON 형식으로 제공해줘: Elon Musk"
# LLM이 생성한 JSON 문자열을 파이썬 딕셔너리로 변환하는 파서
json_parser = JsonOutputParser()

# 가상의 LLM 출력 (실제로는 llm.invoke의 결과)
llm_output_string = '{"name": "Elon Musk", "company": ["SpaceX", "Tesla"], "known_for": ["electric cars", "reusable rockets"]}'

# 파서를 사용하여 문자열을 딕셔너리로 변환
parsed_output = json_parser.parse(llm_output_string)

print(type(parsed_output))
print(parsed_output['company'])

<class 'dict'>
['SpaceX', 'Tesla']


In [7]:
# LCEL 사용 예시
from langchain_core.output_parsers.string import StrOutputParser

# 앞서 정의한 구성 요소들을 파이프로 연결한다.
chain = prompt_template | llm | StrOutputParser()

# 체인 실행
result = chain.invoke({"topic": "대규모 언어 모델", "language": "영어"})
print(result)

A large language model (LLM) is a type of AI trained on vast amounts of text to predict and generate human-like language. Key points:

- Architecture and training: Most modern LLMs use the transformer architecture and are pretrained on huge text corpora to learn patterns of grammar, facts, and style. They are often later fine-tuned for specific tasks.
- How they work: Given some input text, they predict the most likely next words (autoregressive models) or fill in masked words (masked-language models), enabling generation, completion, and understanding of language.
- Capabilities: They can write and summarize text, translate languages, answer questions, generate code, and help with brainstorming or tutoring.
- Limitations and risks: They can produce incorrect or misleading information (“hallucinations”), reflect biases present in training data, and may leak sensitive information. They do not possess true understanding or consciousness.
- Practical use and safety: Effective use often co

In [8]:
# Retrieval 구성 요소 사용 예시
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from google.colab import userdata

# 1. Load: 웹 페이지 로더로 '대형 언어 모델' 위키피디아 문서 로드
loader = WebBaseLoader("https://ko.wikipedia.org/wiki/%EB%8C%80%ED%98%95_%EC%96%B8%EC%96%B4_%EB%AA%A8%EB%8D%B8")
docs = loader.load()

# 2. Split: 텍스트 분할기로 문서를 500자 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 3. Embed & Store: OpenAI 임베딩 모델과 FAISS 벡터 저장소 생성
# 분할된 문서들은 벡터로 변환되어 FAISS에 저장된다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

# 4. Retrieve: 검색기(Retriever)를 생성하고 관련 문서 검색
retriever = vectorstore.as_retriever()
retrieved_docs = retriever.invoke("LLM의 문제점은 무엇인가?")

# 검색된 문서의 내용 중 첫 번째 문서의 일부를 출력
print(retrieved_docs[0].page_content[:200]) # 출력 내용이 길어 일부만 표시

로봇공학, 소프트웨어 공학, 사회적 영향 연구 등의 컴퓨터 과학의 여러 하위 분야에서 LLM의 사용 증가로 이어졌다.[16] 2024년에는 오픈AI가 논리적 추론 기능을 강화한 모델 OpenAI o1을 출시했으며, 이 모델은 최종 답변을 생성하기 전에 장문의 사고 과정(Chains of Thought)을 먼저 생성하는 특징이 있다.
